First all necessary libraries are imported

In [1]:
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go
import chart_studio.plotly as py
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import Dash, Input, Output, callback, dash_table
from plotly.subplots import make_subplots
import dash_bootstrap_components as dbc
import seaborn as sns

# Data preperation and cleaning
As a first step the data is read in as a csv file.

In [2]:
data = pd.read_csv("Testdaten_Generali_all_tested_systems.csv", sep = ";")

As we want to be able to add various different data, we change the column names, so that we can assure, they stay the same. The old names of the columns will vary from data to data and will need to be adapted. 

In [4]:
data = data.rename(columns={"ANWENDUNG":"system", "TESTFALLNAME":"testname", "RESULTAT":"result", "DATUM":"date", "DURCHFÜHRUNGSDAUER":"runtime"})
data.head()

,testname,result,runtime,date,system
0,RRS_VL-Überprüfen(GesperrteAL-WeiterVerteilen),OK,2022-04-12-00.03.47.000000,2022-04-12-17.13.18.000000,EMA Basis
1,RRS_VL-Überprüfen(GesperrteAL-NichtVerteilen),OK,2022-04-12-00.02.45.000000,2022-04-12-17.09.30.000000,EMA Basis
2,RRS_VL-Überprüfen(GesperrteAL-AnStvVerteilen),OK,2022-04-12-00.04.14.000000,2022-04-12-17.06.44.000000,EMA Basis
3,Init FRVIT/VITFRAINT,OK,2022-04-12-00.00.04.000000,2022-04-12-17.02.30.000000,EMA Basis
4,"RRS_AuswertRT-Überprüfen(KMA-oV) [AT,Vitality]",OK,2022-04-12-00.02.11.000000,2022-04-12-17.01.12.000000,EMA Basis


For performance and data seecurity reasons, some tested systems will be removed from the data set and won't be visualized. Also, all tested systems that have less than 5000 rows of data are not included. 

In [14]:
remove_system = ['DM', 'TCM', 'EMA Basis', 'Betriebliche Vorsorge', 'eVtg', 'EVS', 'VISLife AUT Neu']

for i in data['system'].unique():
    if (len(data.loc[(data['system'] == i)]) < 5000):
        remove_system.append(i)

data = data[data['system'].isin(remove_system) == False]
#len(data['system'].unique())
data['system'].unique()

array(['SLS', 'VIS Portal', 'PA', 'ERV'], dtype=object)

All null values are droped from the data

In [15]:
data = data.dropna()

The columns date and runtime need to be adapted, to change the date type to type datetime

In [16]:
count = 0

for index, row in data.iterrows():
    row['date'] = row['date'][:-16]
    row['runtime'] = row ['runtime'][11:]
    count += 1

The result column is transformed into a boolean type. All results that are not "OK" are put together in one category. 

In [17]:
data['result'].loc[(data['result'] != 'OK')] = 0
data['result'].loc[(data['result'] == 'OK')] = 1

The data types are ajusted

In [18]:
data = data.astype({"result": "bool", "date":"datetime64"})
data["runtime"] = pd.to_datetime(data["runtime"], format= "%H.%M.%S.%f").dt.time
data["date"] = pd.to_datetime(data["date"], format= "%Y-%M-%D").dt.date
data.head()

,testname,result,runtime,date,system
228,[GA] Externer Gutachten mit Begründungstext,True,00:04:37,2022-04-12,SLS
235,[Init] Speicherdatei SLS REG,True,00:00:00,2022-04-12,SLS
336,[GATE] Config_GATE_Teambenachrichtigung,True,00:00:00,2022-04-12,VIS Portal
337,[Postcon] Telefonwizard schließen,True,00:00:16,2022-04-12,VIS Portal
339,[Postcon] SLS schließen,True,00:00:08,2022-04-12,VIS Portal


After the data is read in, we want to check, if all data types are set correct. As this is not the case, we set the correct types

In [19]:
data.dtypes

testname    object
result        bool
runtime     object
date        object
system      object
dtype: object

In order to make the table more readable, we use symbols for passed/failed tests

In [20]:
data['result'].loc[(data['result'] == True)] = '🔵 Passed'
data['result'].loc[(data['result'] == False)] = '🔴 Failed'
data.head()

c:\Users\gaert\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,testname,result,runtime,date,system
228,[GA] Externer Gutachten mit Begründungstext,🔵 Passed,00:04:37,2022-04-12,SLS
235,[Init] Speicherdatei SLS REG,🔵 Passed,00:00:00,2022-04-12,SLS
336,[GATE] Config_GATE_Teambenachrichtigung,🔵 Passed,00:00:00,2022-04-12,VIS Portal
337,[Postcon] Telefonwizard schließen,🔵 Passed,00:00:16,2022-04-12,VIS Portal
339,[Postcon] SLS schließen,🔵 Passed,00:00:08,2022-04-12,VIS Portal


A sub dataset is created, that will be visualized in the data table

In [21]:
table_data = data[["system","testname", "result", "date", "runtime"]]
table_data.head()

,system,testname,result,date,runtime
228,SLS,[GA] Externer Gutachten mit Begründungstext,🔵 Passed,2022-04-12,00:04:37
235,SLS,[Init] Speicherdatei SLS REG,🔵 Passed,2022-04-12,00:00:00
336,VIS Portal,[GATE] Config_GATE_Teambenachrichtigung,🔵 Passed,2022-04-12,00:00:00
337,VIS Portal,[Postcon] Telefonwizard schließen,🔵 Passed,2022-04-12,00:00:16
339,VIS Portal,[Postcon] SLS schließen,🔵 Passed,2022-04-12,00:00:08


# Dashboard creation
Next we start building the dashboard.

In [29]:
app = Dash(__name__)

In [30]:
#Basic Design Layout

app.layout = html.Div([

    html.Div([
        html.H1("Software test results", style={'text-align': 'center'}),
    ], style={
        'padding-bottom': '1%',
        'padding-top': '1%',
        "background" : "#EAEAEA",
        "box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"
        }),
    html.Br(),
    html.Br(),
    html.Div(children=[
        
        # Datatable design definition
        html.Div([
            html.Br(),
            html.H3('Details on conducted tests: ' , style = {'text-align': 'center'}),
            html.Br(),
            dash_table.DataTable(
                data = data.to_dict('records'), 
                id="datatable",
                columns= [
                    {"name": i, "id": i} 
                    if i == "testname" or i == "result" or i == "date" or i == "runtime"
                    else {"name": i, "id": i}
                    for i in table_data.columns],
                style_cell={
                    'textAlign': 'left',
                    'padding': '5px',
                    'font-family': 'Arial, Helvetica, sans-serif',
                    },
                style_header = {
                    'fontWeight': 'bold',
                    'border' : '1px solid black',
                    'backgroundColor' : 'rgb(224, 224, 224)',
                    'overflow': 'hidden',
                    'textOverflow': 'ellipsis',
                    'maxWidth': 0,
                },
                style_data_conditional=[
                    {
                        'if' : {'row_index' : 'odd'},
                        'backgroundColor' : 'rgb(242, 243, 245)'
                    },
                ],
                style_data={               
                    'whiteSpace': 'normal',
                    'height': 'auto',
                },
                sort_action="native",
                page_action='native',
                page_size=20, 
                style_table={'height': 1500, 'overflowY': 'auto'},
                fixed_rows={'headers': True},
                filter_action= 'native', 
                 ),
        ], style = {
            'border': 'solid black 1.5px',
            'background' : '#5a6c8a',
            'display': 'float',
            'float' : 'right',
            'padding-left' : '7%',
            'padding-right' : '2%',
            'padding-bottom' :'3%',
            "width": "96%",
            "maxWidth" : "98%",
            "box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"
            }),      
        html.Br(),

    # Dropdowns for the Boxplots design definition

        html.Div([html.P(),
            html.H4('Date or Period of time 1'),
                dcc.Dropdown(id='boxplot_choice1', multi=False, clearable=False,
                        options=[{'label': i, 'value': i} for i in
                        data["date"].unique()],
                        value= data["system"].unique()[0],
                        searchable = True,
                        style = {"box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"}
                                ),
        ], style={
            "width": "46%", 
            'display': 'inline-block',
            'padding-bottom' : '5%',
            }),

        html.Div([html.P(),
            html.H4('Date or Period of time 2'),
                dcc.Dropdown(id='boxplot_choice2', multi=False, clearable=False,
                        options=[{'label': i, 'value': i} for i in
                        data["date"].unique()],
                        searchable = True,
                        value= data["system"].unique()[0],
                        style = {"box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"}
                                ),
                            
        ], style={
            "width": "46%", 
            'display': 'inline-block', 
            'padding-left': '5%',
            'padding-bottom' : '3%',
            'margin-left': '0',
            'margin-right': 'auto',
            }),

     # Boxplots design definition
     html.Div([
        html.H3('Runtime per selected data frame ' , style = {'text-align': 'center'}),
        dcc.Graph(id='boxplot'),
     ], style = {
            'border': 'solid black 1.5px',
            'background' : '#5a6c8a',
            'display': 'float',
            'float' : 'right',
            'padding-left' : '7%',
            'padding-right' : '2%',
            'padding-bottom' :'3%',
            "width": "96%",
            "box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"
            }
        ),

    ], style={
        "width": "53%",
        'font-family': 'Arial, Helvetica, sans-serif',
        'padding-left': '4%', 
        'display': 'float', 
        'float' : 'left',
        'padding-top': '1%',
        'margin-left': '0',
        'margin-right': 'auto',
        }),

   # Number of conducted tests design definition
    html.Div(children=[

        # Dropdown menue for the tested system
        html.Div([
            dcc.Dropdown(
                id='dropdown', multi=True, clearable=False,
                options=[{'label': i, 'value': i} for i in data["system"].unique()],
                style = {
                    "width": "100%",
                    "box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"
                    },
                placeholder="Select the systems you want to see details on...",
                #value= data["system"].unique()[0]
                        ),
        ], style = {
            'display': 'inline-block', 
            'padding-right': '5%',
            'padding-bottom' : '5%',
            'margin-left': '0',
            'margin-right': 'auto',
            'width' : '100%',
        }),

        html.Div([
            html.Br(),
            html.H3('Number of conducted tests: ' , style = {'text-align': 'center'}),
            html.P(
            id='test_number',
            style = {'text-align': 'center', 'font-size' : '3.5em'},
            )
        ], style = {
            'border': 'solid black 1.5px',
            'background' : '#5a6c8a',
            "box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"
            }
        ),
        html.Br(),
        html.Br(),

    # Piechart design definition
        html.Div([
            html.Br(),
            html.H3('Percentage of passed and failed tests: ' , style = {'text-align': 'center'}),
            dcc.Graph(id='piechart', style={"height": "10%"}),
        ], style = {
            'border': 'solid black 1.5px',
            'background' : '#5a6c8a',
            "box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"
            }
        ),
        html.Br(),
        html.Br(),

    # Linechart design definition
        html.Div([
            html.Br(),
            html.H3('Passed test rate over time: ' , style = {'text-align': 'center'}),
            dcc.Graph(id='linechart')
        ], style = {
            'border': 'solid black 1.5px',
            'background' : '#5a6c8a',
            
            },
        ),

    ],style={
        "width": "40%", 
        'display': 'float',
        'float' : 'right',
        'font-family': 'Arial, Helvetica, sans-serif',
        'padding-top': '1%',
        'margin-left': 'auto',
        'margin-right': '0',
    }),
    
], style={
    "padding": "10px",
    'font-family': 'Arial, Helvetica, sans-serif',
})
 

In [31]:
@app.callback(

    #Output for the visualization and Inputs from the visualization    
    [Output(component_id='test_number', component_property='children'),
    Output(component_id='boxplot', component_property='figure'),
    Output(component_id='piechart', component_property='figure'),
    Output(component_id='linechart', component_property='figure')],
    [Input(component_id='dropdown', component_property='value'),
    Input(component_id='boxplot_choice1', component_property='value'),
    Input(component_id='boxplot_choice2', component_property='value'),
    Input(component_id='datatable', component_property="derived_virtual_data")]
)

def update_data(option_slctd, date_choice1, date_choice2, all_rows_data):

    # Defining the three data tables on which the graphs are built upon
    # dff is the first sub-data table that takes in consideration, which filter are active in the visualization
    dff = pd.DataFrame(all_rows_data)

    if option_slctd == None:
        data_filtered = dff
    else:
        if len(option_slctd) != 0:
            data_filtered = dff.loc[dff['system'].isin(option_slctd)]
        else:
            data_filtered = dff

    # boxplot_data is the second sub-data table used for the boxplots, as they need another time/date format
    boxplot_data = data_filtered
    boxplot_data['runtime'] = pd.to_datetime(data['runtime'], format= "%H:%M:%S%f", infer_datetime_format=True)

    # dff_sub is the third sub-data table used for the piechart visualization
    dff_sub= data_filtered.groupby('result').size().reset_index(name='count')

    # Calculations for the % of passed tests per day   
    y = data_filtered.loc[(data_filtered['result'] == '🔵 Passed')]
    passed_per_day=[]
    grouped_per_day = []
    group_size = 3

    for j in data['system'].unique():
        x = y.loc[(data_filtered['system']) == j]
        z = data_filtered.loc[(data_filtered['system']) == j]
        for i in data_filtered['date'].unique():
            if len(z.loc[data_filtered['date'] == i]) == 0:
                continue
            else:
                passed_per_day.append((len(x.loc[data_filtered['date'] == i]))/len(z.loc[data_filtered['date'] == i])*100)
                passed_per_day.append(str(i))
                passed_per_day.append(str(j))

    for i in range (0, len(passed_per_day), group_size):
        grouped_per_day.append(passed_per_day[i:i+group_size]) 

    data_passed_percentage = pd.DataFrame(grouped_per_day, columns = ['passed%', 'date', 'system'])

    number_of_tests = len(data_filtered)

    # Building the boxplots
    trace0 = go.Box(
        y=boxplot_data.loc[boxplot_data['date'] == str(date_choice1)]['runtime'].sort_values(), 
        name = date_choice1,
        marker = dict(
        color = 'rgb(44, 39, 117)'
        ),
        hoverinfo='none'
    )
    
    trace1 = go.Box(
        y= boxplot_data.loc[boxplot_data['date'] == str(date_choice2)]['runtime'].sort_values(),
        name = date_choice2,
        marker = dict(
        color = 'rgb(50, 173, 155)'
        ),
        hoverinfo='none'
    )
    data1 = [trace0,trace1]
    layout = go.Layout(  
        template='ggplot2'
    )

    box_plot = go.Figure(data=data1,layout=layout)
    box_plot.update_layout(paper_bgcolor="#5a6c8a")

    # Building the piechart
    pie_chart=px.pie(
            data_frame=dff_sub,
            names='result',
            color= 'result',
            values='count',
            hole=.3,
            color_discrete_map= {'🔵 Passed':'royalblue', '🔴 Failed' : '#de071d'},
            labels={'result':'count'}
            )
    pie_chart.update_layout(paper_bgcolor="#5a6c8a")
    # Building the line chart
    line_chart = px.line(
            data_frame=data_passed_percentage,
            x='date',
            y='passed%',
            color= 'system'
            )
    line_chart.update_layout(hovermode="x unified", paper_bgcolor="#5a6c8a")
    line_chart.update_xaxes(rangeslider_visible=True)
    line_chart.update_traces(mode="markers+lines")

    return (number_of_tests, box_plot, pie_chart, line_chart)


In [32]:
if __name__ == '__main__':
    app.run_server(debug=False)
    

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Oct/2022 22:19:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2022 22:19:52] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_3_0m1648480726.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2022 22:19:52] "GET /_dash-component-suites/dash/deps/react@16.v2_3_0m1648480726.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2022 22:19:52] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2022 22:19:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2022 22:19:53] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2022 22:19:53] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Oct/2022 22:19:53] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2022 22:19:53] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "c:\Users\gaert\anaconda3\lib\site-packages\dash\dash.py", line 1345, in dispatch
    response.set_d

127.0.0.1 - - [04/Oct/2022 22:19:53] "POST /_dash-update-component HTTP/1.1" 500 -
C:\Users\gaert\AppData\Local\Temp/ipykernel_1140/4245666055.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

127.0.0.1 - - [04/Oct/2022 22:19:57] "POST /_dash-update-component HTTP/1.1" 200 -
